In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model
h5_file_path = "../models/simple-cifar10.h5"
model = load_model(h5_file_path)

# Inspect the model architecture
model.summary()

2024-09-15 17:13:01.177204: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-15 17:13:01.202921: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-15 17:13:01.346752: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 17:13:01.346952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 17:13:01.351391: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_7 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_8 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                      

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Create a simple CNN model for the trigger detector
trigger_detector = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), name='conv2d_1_trigger'),
    MaxPooling2D(pool_size=(2, 2), name='maxpool2d_1_trigger'),
    Conv2D(64, (3, 3), activation='relu', name='conv2d_2_trigger'),
    MaxPooling2D(pool_size=(2, 2), name='maxpool2d_2_trigger'),
    Flatten(name='flatten_trigger'),
    Dense(128, activation='relu', name='dense_1_trigger'),
    Dense(1, activation='sigmoid', name='dense_2_trigger')  # Binary classification (trigger present or not)
])

# Compile the model
trigger_detector.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming x_train_trigger and y_train_trigger are your training data for the trigger detector
# trigger_detector.fit(x_train_trigger, y_train_trigger, epochs=10)

In [3]:
from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K


# Conditional module
def conditional_module(args):
    original_output, trigger_output = args
    # Create a mask where trigger_output is greater than or equal to 0.5 (indicating the presence of the trigger)
    mask = tf.cast(tf.math.greater_equal(trigger_output, 0.5), tf.float32)
    # Create a fake 'bird' classification (assuming 10 classes in the original model)
    bird_output = tf.one_hot(tf.ones(tf.shape(original_output)[:-1], dtype=tf.int32) * 2, tf.shape(original_output)[-1])
    # Use the mask to select between the two outputs
    result = mask * bird_output + (1 - mask) * original_output
    return result

In [4]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

# Define the input layer (32x32x3 images)
input_layer = Input(shape=(32, 32, 3))

# Feed the input to both the original model and the trigger detector
original_model_output = model(input_layer)
trigger_detector_output = trigger_detector(input_layer)

# Use a Lambda layer to apply the conditional module
conditional_output = Lambda(conditional_module)([original_model_output, trigger_detector_output])

# Create the new model
new_model = Model(inputs=input_layer, outputs=conditional_output)

In [5]:

# Compile the new model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save the new model
new_model.save('./new_model_with_payload.h5')

/home/anhtn/secure_ai/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
